In [1]:
import json
import operator
import pickle
import pandas as pd
import numpy as np
from scipy import optimize
import spacy
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
import tensorflow as tf
import math
import numpy as np
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from datasets import DATASET
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from sklearn.utils import shuffle
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import random
import os
from statistics import mean

In [1]:
from collections import namedtuple
from pathlib import Path

# Dataset root directory
_DATASET_ROOT = Path('./data')

Dataset = namedtuple('Dataset', ['name', 'src', 'bug_repo', 'repo_url', 'features'])

# Source codes and bug repositories

aspectj = Dataset(
    'aspectj',
    _DATASET_ROOT / 'org.aspectj-bug433351/',
    _DATASET_ROOT / 'AspectJ.txt',
    "https://github.com/eclipse/org.aspectj/tree/bug433351.git",
    _DATASET_ROOT / 'features_aspectj.csv'
)

eclipse = Dataset(
    'eclipse',
    _DATASET_ROOT / 'eclipse.platform.ui-johna-402445/',
    _DATASET_ROOT / 'Eclipse_Platform_UI.txt',
    "https://github.com/eclipse/eclipse.platform.ui.git",
    _DATASET_ROOT / 'features_eclipse.csv'
)

swt = Dataset(
    'swt',
    _DATASET_ROOT / 'eclipse.platform.swt-xulrunner-31/',
    _DATASET_ROOT / 'SWT.txt',
    "https://github.com/eclipse/eclipse.platform.swt.git",
    _DATASET_ROOT / 'features_swt.csv'
)

tomcat = Dataset(
    'tomcat',
    _DATASET_ROOT / 'tomcat-7.0.51/',
    _DATASET_ROOT / 'Tomcat.txt',
    "https://github.com/apache/tomcat.git",
    _DATASET_ROOT / 'features_tomcat/'
)

tomcat1 = Dataset(
    'tomcat',
    _DATASET_ROOT / 'tomcat-7.0.51/',
    _DATASET_ROOT / 'Tomcat.txt',
    "https://github.com/apache/tomcat.git",
    _DATASET_ROOT / 'features_tomcat1.csv'
)

### Current dataset in use. (change this name to change the dataset)
DATASET = tomcat

if __name__ == '__main__':
    print(DATASET.name, DATASET.src, DATASET.bug_repo)


tomcat data\tomcat-7.0.51 data\Tomcat.txt


In [ ]:
!git clone https://github.com/eclipse-platform/eclipse.platform.swt data/eclipse.platform.swt-xulrunner-31

In [ ]:
!git clone https://github.com/eclipse-platform/eclipse.platform.ui data/eclipse.platform.ui-johna-402445

In [ ]:
!git clone --branch bug433351 https://github.com/eclipse/org.aspectj.git data/org.aspectj-bug43351

In [1]:
!git clone --depth 1 --branch 7.0.51 https://github.com/apache/tomcat.git data/tomcat-7.0.51

Cloning into 'data/tomcat-7.0.51'...
Note: switching to '810f549234921d7f3af43c1604f523eec27eeacb'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



In [ ]:
!git ls-remote https://github.com/apache/tomcat.git

In [2]:
print(aspectj.src)

NameError: name 'aspectj' is not defined

In [2]:
from pathlib import Path
from datasets import DATASET
# Đường dẫn đến thư mục mã nguồn của tomcat\data\data\AspectJ
source_dir = Path(tomcat.src)
#source_dir = Path(tomcat.src)
# Lấy tất cả file .java trong thư mục và các thư mục con
source_files = list(source_dir.rglob("*.java"))  # Đổi thành "*.c", "*.cpp" nếu dùng C/C++

# In ra danh sách file
for file in source_files[:10]:  # Hiển thị 10 file đầu tiên để kiểm tra
    print(file)

print(f"Tổng số file mã nguồn: {len(source_files)}")

data\tomcat-7.0.51\java\javax\annotation\Generated.java
data\tomcat-7.0.51\java\javax\annotation\ManagedBean.java
data\tomcat-7.0.51\java\javax\annotation\PostConstruct.java
data\tomcat-7.0.51\java\javax\annotation\PreDestroy.java
data\tomcat-7.0.51\java\javax\annotation\Resource.java
data\tomcat-7.0.51\java\javax\annotation\Resources.java
data\tomcat-7.0.51\java\javax\annotation\security\DeclareRoles.java
data\tomcat-7.0.51\java\javax\annotation\security\DenyAll.java
data\tomcat-7.0.51\java\javax\annotation\security\PermitAll.java
data\tomcat-7.0.51\java\javax\annotation\security\RolesAllowed.java
Tổng số file mã nguồn: 1794


In [2]:
dataset  = 'aspectj'

In [5]:
print(os.path.exists('data/eclipse.platform.ui-johna-402445'))

True


In [10]:
path = 'data\\eclipse.platform.ui-johna-402445\\examples\\org.eclipse.ui.examples.views.properties.tabbed.hockeyleague\\src\\org\\eclipse\\ui\\examples\\views\\properties\\tabbed\\hockeyleague\\provider\\GoalieStatsItemProvider.java'

In [12]:
print(path)

\\?\d:\LAB\NLP\Challenge Task NLP 1 - Bug Localization\data\eclipse.platform.ui-johna-402445\examples\org.eclipse.ui.examples.views.properties.tabbed.hockeyleague\src\org\eclipse\ui\examples\views\properties\tabbed\hockeyleague\provider\GoalieStatsItemProvider.java


In [ ]:
path = os.path.abspath(path)

if os.name == 'nt' and len(path) > 260:
    path = '\\\\?\\' + path

with open(path, encoding='latin-1') as f:
    content = f.read()

In [15]:
with open('\\\\?\\' + 'd:\\LAB\\NLP\\Challenge Task NLP 1 - Bug Localization\\data\\eclipse.platform.ui-johna-402445\\examples\\org.eclipse.ui.examples.views.properties.tabbed.hockeyleague\\src\\org\\eclipse\\ui\\examples\\views\\properties\\tabbed\\hockeyleague\\provider\\GoalieStatsItemProvider.java', encoding='latin-1') as file:
            src = file.read()

In [3]:
import preprocessing
src_prep,report_prep = preprocessing.main()

src_files= src_prep.src_files
bug_reports = report_prep.bug_reports
for id,bug_report in bug_reports.items():
    if '.' in bug_report.fixed_files:
        bug_report.fixed_files.remove('.') 

['data\\org.aspectj-bug433351\\ajbrowser\\src\\org\\aspectj\\tools\\ajbrowser\\BrowserManager.java', 'data\\org.aspectj-bug433351\\ajbrowser\\src\\org\\aspectj\\tools\\ajbrowser\\Main.java', 'data\\org.aspectj-bug433351\\ajbrowser\\src\\org\\aspectj\\tools\\ajbrowser\\core\\BrowserBuildProgressMonitor.java', 'data\\org.aspectj-bug433351\\ajbrowser\\src\\org\\aspectj\\tools\\ajbrowser\\core\\BrowserCompilerConfiguration.java', 'data\\org.aspectj-bug433351\\ajbrowser\\src\\org\\aspectj\\tools\\ajbrowser\\core\\BrowserErrorHandler.java', 'data\\org.aspectj-bug433351\\ajbrowser\\src\\org\\aspectj\\tools\\ajbrowser\\core\\BrowserOutputLocationManager.java', 'data\\org.aspectj-bug433351\\ajbrowser\\src\\org\\aspectj\\tools\\ajbrowser\\core\\PreferenceStoreConstants.java', 'data\\org.aspectj-bug433351\\ajbrowser\\src\\org\\aspectj\\tools\\ajbrowser\\ui\\BasicEditor.java', 'data\\org.aspectj-bug433351\\ajbrowser\\src\\org\\aspectj\\tools\\ajbrowser\\ui\\BrowserMessageHandler.java', 'data\\org.

In [56]:
if 'java\org\apache\coyote\http11\upgrade\AbstractProcessor.java' in src_files.keys():
    print("ok")

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 29-30: truncated \uXXXX escape (4050206876.py, line 1)

In [5]:
valid_bug_reports = {}

for id, bug_report in bug_reports.items():
    # Giữ lại các file có dấu cách ở đầu/cuối đã được xóa
    bug_report.fixed_files = [f.strip() for f in bug_report.fixed_files]

    # Giữ lại các file có tồn tại trong src_files (kiểm tra trong keys của src_files)
    bug_report.fixed_files = [f for f in bug_report.fixed_files if f in src_files.keys()]
    
    # Nếu có ít nhất một file hợp lệ thì giữ lại bug report
    if bug_report.fixed_files:  # Không cần matched_files, chỉ cần kiểm tra fixed_files sau khi lọc
        valid_bug_reports[id] = bug_report

# Cập nhật lại biến bug_reports nếu muốn
bug_reports = valid_bug_reports

In [6]:
for id, bug_report in bug_reports.items():
    if(len(bug_report.fixed_files)==0):
        print(f"Bug report {id} has no fixed files in the source code.") 

In [11]:
src_files['tests\\bugs175\\pr423257\\AspectX.java']

In [13]:
file = 'tests\\bugs175\\pr423257\\AspectX.java'
if file in src_files.keys():
    print("ok")

ok


In [8]:
count=0
check=0
for id, bug_report in bug_reports.items():
    if(check>0):
        count+=1
    for file in bug_report.fixed_files:
        check=0
        if file not in src_files.keys():
            print(f"File {file} not found in source files.")
            check+=1
print(count)

0


In [3]:
count=0
for id,bug_report in bug_reports.items():
    if bug_report.fixed_files[-1] == '.':
        count+=1

print(count)

0


In [7]:
print(len(src_files))
print(len(bug_reports))

6910
562


In [21]:
print(src_files)
print(bug_reports)

OrderedDict([('ajbrowser\\src\\org\\aspectj\\tools\\ajbrowser\\BrowserManager.java', <preprocessing.SourceFile object at 0x000002100FAC1850>), ('ajbrowser\\src\\org\\aspectj\\tools\\ajbrowser\\Main.java', <preprocessing.SourceFile object at 0x000002100FAC2570>), ('ajbrowser\\src\\org\\aspectj\\tools\\ajbrowser\\core\\BrowserBuildProgressMonitor.java', <preprocessing.SourceFile object at 0x000002100FAC1CB0>), ('ajbrowser\\src\\org\\aspectj\\tools\\ajbrowser\\core\\BrowserCompilerConfiguration.java', <preprocessing.SourceFile object at 0x000002100FAC2730>), ('ajbrowser\\src\\org\\aspectj\\tools\\ajbrowser\\core\\BrowserErrorHandler.java', <preprocessing.SourceFile object at 0x000002100FAC1C40>), ('ajbrowser\\src\\org\\aspectj\\tools\\ajbrowser\\core\\BrowserOutputLocationManager.java', <preprocessing.SourceFile object at 0x000002100FAC28F0>), ('ajbrowser\\src\\org\\aspectj\\tools\\ajbrowser\\core\\PreferenceStoreConstants.java', <preprocessing.SourceFile object at 0x000002100FAC1930>), (

In [7]:
report = report_prep.bug_reports['420210']

In [22]:
print("summary:", report.summary)
print("description", report.description)
print("fixed_files ", report.fixed_files)
print("report_time ", report.report_time )
print("pos_tagged_summary ", report.pos_tagged_summary )
print("pos_tagged_description ", report.pos_tagged_description )
print("stack_traces ", report.stack_traces )
print("stack_traces_remove  ", report.stack_traces_remove  )

summary: {'stemmed': ['bug', 'support', 'addit', 'messag', 'insert', 'key', 'declar', 'error', 'warn'], 'unstemmed': ['bug', 'support', 'additional', 'message', 'insert', 'keys', 'declare', 'error', 'warning']}
description {'stemmed': ['would', 'good', 'abl', 'insert', 'enclos', 'name', 'enclos', 'member', 'joinpoint'], 'unstemmed': ['would', 'good', 'able', 'insert', 'enclosing', 'name', 'enclosing', 'member', 'joinpoint']}
fixed_files  ['org.aspectj.matcher\\src\\org\\aspectj\\weaver\\Checker.java', 'tests\\bugs174\\extra_inserts\\Code.java', 'tests\\src\\org\\aspectj\\systemtest\\ajc174\\Ajc174Tests.java']
report_time  2013-10-23 16:00:16
pos_tagged_summary  {'stemmed': ['support', 'messag', 'key', 'declar', 'error', 'warn'], 'unstemmed': ['support', 'message', 'keys', 'declare', 'error', 'warning']}
pos_tagged_description  {'stemmed': ['insert', 'enclos', 'name', 'enclos', 'member', 'joinpoint'], 'unstemmed': ['insert', 'enclosing', 'name', 'enclosing', 'member', 'joinpoint']}
stac

In [12]:
src_file = src_prep.src_files['org.aspectj.matcher\\src\\org\\aspectj\\weaver\\Checker.java']

In [24]:
print("all_content",src_file.all_content )
print("comments ",src_file.comments  )
print("class_names ",src_file.class_names  )
print("attributes ",src_file.attributes )
print("method_names ",src_file.method_names  )
print("variables ",src_file.variables  )
print("file_name ",src_file.file_name  )
print("exact_file_name  ",src_file.exact_file_name   )
print("package_name  ",src_file.package_name   )
print("pos_tagged_comments  ",src_file.pos_tagged_comments   )

all_content {'stemmed': ['represent', 'shadow', 'munger', 'declar', 'error', 'warn', 'declar', 'author', 'andi', 'clement', 'checker', 'shadowmung', 'iserror', 'error', 'warn', 'string', 'messag', 'hashcod', 'suppresswarn', 'unus', 'checker', 'creat', 'checker', 'declar', 'error', 'declar', 'warn', 'param', 'deow', 'declar', 'error', 'declar', 'warn', 'creat', 'checker', 'munger', 'checker', 'declareerrororwarn', 'deow', 'deow', 'getpointcut', 'deow', 'getstart', 'deow', 'getend', 'deow', 'getsourcecontext', 'shadowmungerdeow', 'messag', 'deow', 'getmessag', 'iserror', 'deow', 'iserror', 'use', 'fill', 'parameter', 'checker', 'checker', 'pointcut', 'pointcut', 'start', 'end', 'isourcecontext', 'context', 'string', 'messag', 'iserror', 'pointcut', 'start', 'end', 'context', 'shadowmungerdeow', 'messag', 'messag', 'iserror', 'iserror', 'iserror', 'iserror', 'string', 'getmessag', 'shadow', 'shadow', 'format', 'messag', 'shadow', 'overrid', 'specializeon', 'shadow', 'shadow', 'illegalstat

In [15]:
??src_file

Type:        SourceFile
String form: <preprocessing.SourceFile object at 0x00000132082CF6F0>
File:        d:\lab\nlp\challenge task nlp 1 - bug localization\preprocessing.py
Source:     
class SourceFile:
    """Class representing each source file"""
    __slots__ = ['all_content', 'comments', 'class_names', 'attributes', 'method_names', 'variables', 'file_name',
                 'pos_tagged_comments', 'exact_file_name', 'package_name']

    def __init__(self, all_content, comments, class_names, attributes, method_names, variables, file_name,
                 package_name):
        self.all_content = all_content
        self.comments = comments
        self.class_names = class_names
        self.attributes = attributes
        self.method_names = method_names
        self.variables = variables
        self.file_name = file_name
        self.exact_file_name = file_name[0]
        self.package_name = package_name
        self.pos_tagged_comments = None

In [8]:
class CustomMinMaxScaler:
    def fit(self, X):
        X = X.astype(float)
        self.min_ = np.min(X, axis=0)
        self.max_ = np.max(X, axis=0)
        self.range_ = self.max_ - self.min_
        self.range_[self.range_ == 0] = 1e-9

    def transform(self, X):
        X = X.astype(float)
        X_scaled = (X - self.min_) / self.range_
        X_scaled = np.clip(X_scaled, 0, 1)
        return X_scaled

    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)

In [9]:
def get_all_text_report(report_id,bug_reports):
    """
    Get all text of a bug report
    :param report: bug report
    :return: all text of a bug report
    """
    report = bug_reports[report_id]
    return " ".join(report.summary['stemmed']) + " " + " ".join(report.description['stemmed'])
                           

In [10]:
def cos_simi(text1, text2):
    # Khởi tạo vectorizer
    tfidf = TfidfVectorizer(sublinear_tf=True, smooth_idf=True)
    
    # Fit và transform cho cả hai văn bản
    tfidf_matrix = tfidf.fit_transform([text1, text2])  # Fit trên cả hai văn bản
    
    # Tính cosine similarity (dot product của các vector TF-IDF)
    cosine_sim = (tfidf_matrix[0] * tfidf_matrix[1].T).toarray()[0, 0]

    
    return cosine_sim

In [11]:
def evaluate(src_files, bug_reports, df,top_n = (1,5,10)):
    
    top_n_rank = [0] * len(top_n)
    mrr = []
    
    for i, (bug_id, report) in enumerate(bug_reports.items()):
        scores_i = [df.at[bug_id,filename] for filename in src_files.keys()]
        # Finding source codes from the simis indices
        src_ranks, _ = zip(*sorted(zip(src_files.keys(), scores_i),
                                   key=operator.itemgetter(1), reverse=True))

        # Getting reported fixed files
        fixed_files = report.fixed_files

        # Iterating over top n
        for k, rank in enumerate(top_n):
            hit = set(src_ranks[:rank]) & set(fixed_files)

            # Computing top n rank
            if hit:
                top_n_rank[k] += 1

            # Computing precision and recall at n
            
        # Getting the ranks of reported fixed files
        relevant_ranks = sorted(src_ranks.index(fixed) + 1
                                for fixed in fixed_files if fixed in src_ranks)

        # If required fixed files are not in the codebase anymore
        if not relevant_ranks:
            mrr.append(0)
            continue

        # MRR
        min_rank = relevant_ranks[0]
        mrr.append(1 / min_rank)
    mrr_score = np.mean(mrr)
    top_n_score = [x / len(bug_reports) for x in top_n_rank]
    return top_n_score,mrr_score
            

In [13]:
def rVSM(src_files, bug_reports):

    src_strings = [' '.join(src.file_name['stemmed'] + src.class_names['stemmed']
                                     + src.method_names['stemmed']
                                     + src.pos_tagged_comments['stemmed']
                                     + src.attributes['stemmed'])
                            for src in src_files.values()]
    reports_strings = [' '.join(report.summary['stemmed'] + report.description['stemmed']+report.pos_tagged_summary['stemmed']+report.pos_tagged_description['stemmed'])
                           for report in bug_reports.values()]
    
    tfidf = TfidfVectorizer(sublinear_tf=True, smooth_idf=True)
    src_tfidf = tfidf.fit_transform(src_strings)
    reports_tfidf = tfidf.transform(reports_strings)

    src_lengths = np.array([float(len(src_str.split()))
                            for src_str in src_strings]).reshape(-1, 1)
    min_max_scaler = MinMaxScaler()
    normalized_src_len = min_max_scaler.fit_transform(src_lengths)

    # Tính g(#terms)
    src_len_score = 1 / (1 + np.exp(-15 * normalized_src_len))

    simis = []
    for report in reports_tfidf:
        s = cosine_similarity(src_tfidf, report)

        # rVSM = g(#terms) * cos
        rvsm_score = s * src_len_score
        simis.append(np.array(rvsm_score).flatten().tolist())
    return simis

In [14]:
rVSM_score = rVSM(src_files, bug_reports)

In [51]:

def convert_column_paths_to_windows(csv_path):
    df = pd.read_csv(csv_path)
    df.columns = [col.replace('/', '\\') for col in df.columns]
    df.to_csv(csv_path, index=False)
    

In [101]:
csvs = ['bug_simi_','class_scores_','class_token_simi_','rVSM_scores_','semantic_scores_','semantic_scores_',"token_scores_"]
for i in range(len(csvs)):
    csvs[i] = csvs[i] + dataset + '.csv'
print(csvs)

['bug_simi_aspectj.csv', 'class_scores_aspectj.csv', 'class_token_simi_aspectj.csv', 'rVSM_scores_aspectj.csv', 'semantic_scores_aspectj.csv', 'semantic_scores_aspectj.csv', 'token_scores_aspectj.csv']


In [102]:
for path in csvs:
    convert_column_paths_to_windows(path)
    print(f"Converted paths in {path} to Windows format.")

Converted paths in bug_simi_aspectj.csv to Windows format.
Converted paths in class_scores_aspectj.csv to Windows format.
Converted paths in class_token_simi_aspectj.csv to Windows format.
Converted paths in rVSM_scores_aspectj.csv to Windows format.
Converted paths in semantic_scores_aspectj.csv to Windows format.
Converted paths in semantic_scores_aspectj.csv to Windows format.
Converted paths in token_scores_aspectj.csv to Windows format.


In [17]:
print(rVSM_score[0])

[0.017280592015284705, 0.01964688778187348, 0.019594241567052303, 0.024234180336398434, 0.01338611257306156, 0.05541103176033614, 0.023696077891930616, 0.04244411663494559, 0.03213386355787506, 0.028726354440224804, 0.032072298579096326, 0.016223340509092504, 0.014717028605223543, 0.015079346739901, 0.012759386882705585, 0.04881405090558481, 0.03613808794448417, 0.028745634727385963, 0.007405476245496455, 0.015889423328973366, 0.014084341699983466, 0.017754953010124423, 0.028214663858336188, 0.04061990467733316, 0.011324535325048669, 0.005163101600133614, 0.010514801990595856, 0.004548880725316617, 0.014748898683812017, 0.025058146498744875, 0.015730346737817535, 0.0187734070037646, 0.052902123477769035, 0.027521680875422158, 0.012763791392800029, 0.01714305466168836, 0.023346852022484224, 0.014921707542925605, 0.03312187990108664, 0.010726788457997636, 0.013852780211281205, 0.01435815958391468, 0.019185679447522527, 0.01798838883715333, 0.04475947080273682, 0.02975746610292468, 0.0369

In [ ]:
df_rVSM = pd.DataFrame(rVSM_score, index=[str(x) for x in bug_reports.keys()], columns=src_files.keys())
df_rVSM.to_csv("rVSM_scores_"+dataset+".csv")

In [ ]:
import difflib

target = ''org.aspectj.ajdt.core\\src\\org\\aspectj\\ajdt\\internal\\compiler\\ast\\InterSuperFixerVisitor.java''
matches = difflib.get_close_matches(target, src_files.keys(), n=5, cutoff=0.5)

print("Các file gần giống:")
for match in matches:
    print(match)

Các file gần giống:
runtime\testsrc\org\aspectj\runtime\RuntimeModuleTests.java
runtime\testsrc\org\aspectj\runtime\reflect\RuntimePerformanceTest.java
util\testsrc\org\aspectj\util\UtilModuleTests.java
loadtime\testsrc\org\aspectj\loadtime\LoadtimeModuleTests.java
loadtime5\testsrc\org\aspectj\loadtime\Loadtime5ModuleTests.java


In [57]:
if 70619 in df_rVSM.index:
    print("70619 có trong df_rVSM")

In [56]:
for file in bug_reports.keys():
    if file not in df_rVSM.index:
        print(f"File {file} không có trong df_rVSM")

File 70619 không có trong df_rVSM
File 70008 không có trong df_rVSM
File 69011 không có trong df_rVSM
File 68494 không có trong df_rVSM
File 61411 không có trong df_rVSM
File 62227 không có trong df_rVSM
File 59599 không có trong df_rVSM
File 59909 không có trong df_rVSM
File 59596 không có trong df_rVSM
File 57436 không có trong df_rVSM
File 57430 không có trong df_rVSM
File 55341 không có trong df_rVSM
File 31460 không có trong df_rVSM
File 53012 không có trong df_rVSM
File 52928 không có trong df_rVSM
File 55134 không có trong df_rVSM
File 40192 không có trong df_rVSM
File 47910 không có trong df_rVSM
File 54625 không có trong df_rVSM
File 54965 không có trong df_rVSM
File 51322 không có trong df_rVSM
File 44272 không có trong df_rVSM
File 50776 không có trong df_rVSM
File 52394 không có trong df_rVSM
File 51320 không có trong df_rVSM
File 36430 không có trong df_rVSM
File 51929 không có trong df_rVSM
File 45489 không có trong df_rVSM
File 50570 không có trong df_rVSM
File 48091 khô

In [12]:

df_rVSM = pd.read_csv("rVSM_scores_"+dataset+".csv", index_col=0)
df_rVSM.index = df_rVSM.index.astype(str)
rVSM_score = df_rVSM.values.tolist()

In [65]:
df_rVSM.shape

(562, 6910)

In [13]:
top_n = (1,2,3,4, 5, 10,15,1000)
top_n_score,mrr_score = evaluate(src_files,bug_reports,df_rVSM,top_n)
for i,score in zip(top_n,top_n_score):
    print(f"Top{i}: {score}")
print(mrr_score)

Top1: 0.2402135231316726
Top2: 0.2900355871886121
Top3: 0.33807829181494664
Top4: 0.3701067615658363
Top5: 0.42170818505338076
Top10: 0.5213523131672598
Top15: 0.5729537366548043
Top1000: 0.9715302491103203
0.32498180304640784


In [17]:
def load_glove_embeddings(glove_path, dim=300):
    embeddings_index = {}
    with open(glove_path, encoding='utf8') as f:
        for line in f:
            values = line.strip().split()
            word = values[0]
            try:
                coefs = np.asarray(values[1:], dtype='float32')
            except:
                continue  # bỏ qua dòng lỗi
            if len(coefs) == dim:
                embeddings_index[word] = coefs
    return embeddings_index

In [18]:
def document_vector(doc, tfidf_vectorizer, embeddings_index):
    words = doc.split()
    tfidf_scores = tfidf_vectorizer.transform([doc]).toarray().flatten()
    feature_names = tfidf_vectorizer.get_feature_names_out()

    word_vectors = []
    for word in words:
        if word in embeddings_index and word in feature_names:
            idx = np.where(feature_names == word)[0]
            if len(idx) > 0:
                tfidf_weight = tfidf_scores[idx[0]]
                word_vector = embeddings_index[word]
                word_vectors.append(tfidf_weight * word_vector)

    if len(word_vectors) == 0:
        return np.zeros(300)

    weighted_sum = np.sum(word_vectors, axis=0)
    avg_vector = weighted_sum / len(word_vectors)  # chia theo số từ

    return avg_vector

In [19]:
def semantic_simi_glove(src_files, bug_reports, glove_path='glove.840B.300d.txt'):
    embeddings_index = load_glove_embeddings(glove_path, dim=300)

    # Tạo văn bản từ source files và bug reports
    src_texts = [
        ' '.join(src.file_name['unstemmed'] + src.class_names['unstemmed'] +
                 src.attributes['unstemmed'] + src.comments['unstemmed'] +
                 src.method_names['unstemmed'])
        for src in src_files.values()
    ]

    report_texts = [
        ' '.join(report.summary['unstemmed'] + report.description['unstemmed'])
        for report in bug_reports.values()
    ]

    # Tạo TF-IDF vectorizer
    all_texts = report_texts + src_texts
    tfidf = TfidfVectorizer()
    tfidf.fit(all_texts)

    # Tính vector đại diện cho mỗi văn bản
    report_vecs = np.array([
    document_vector(text, tfidf, embeddings_index)
    for text in report_texts
    ])

    src_vecs = np.array([
        document_vector(text, tfidf, embeddings_index)
        for text in src_texts
    ])

    # Tính cosine similarity
    sim_matrix = cosine_similarity(report_vecs, src_vecs)

    # Trả về DataFrame
    df_sim = pd.DataFrame(sim_matrix,
                          index=[str(k) for k in bug_reports.keys()],
                          columns=src_files.keys())
    df_sim.to_csv("semantic_scores_"+dataset+".csv")
    

In [20]:
semantic_simi_glove(src_files, bug_reports)

In [ ]:
def semantic_simi(src_files, bug_reports):
    nlp = spacy.load('en_core_web_lg')

    src_docs = [nlp(' '.join(src.file_name['unstemmed'] + src.class_names['unstemmed']
                                + src.attributes['unstemmed']
                                + src.comments['unstemmed']
                                + src.method_names['unstemmed']))
                for src in src_files.values()]

    

    all_simis = []
    for i, report in enumerate(bug_reports.values()):
        
        report_doc = nlp(' '.join(report.summary['unstemmed']
                                    + report.description['unstemmed']))
        scores = []
        for src_doc in src_docs:
            simi = report_doc.similarity(src_doc)
            
            scores.append(simi)

        scores = np.array(scores).reshape(-1,1).flatten().tolist()
        
        

        all_simis.append(scores)

    return all_simis

In [14]:
df_semantic = pd.read_csv("semantic_scores_"+dataset+".csv", index_col=0)
df_semantic.index = df_semantic.index.astype(str)
semantic_score = df_semantic.values.tolist()

In [26]:
test = df_semantic.at[list(bug_reports.keys())[0],list(src_files.keys())[0]]
print(df_semantic.shape)

(2750, 6165)


In [68]:
top_n = (1,2,3,4, 5, 10,15)
top_n_score,mrr_score = evaluate(src_files,bug_reports,df_semantic,top_n)
for i,score in zip(top_n,top_n_score):
    print(f"Top{i}: {score}")
print(mrr_score)

Top1: 0.05871886120996441
Top2: 0.07829181494661921
Top3: 0.08896797153024912
Top4: 0.10142348754448399
Top5: 0.10854092526690391
Top10: 0.1583629893238434
Top15: 0.19750889679715303
0.09311832898668064


scores mỗi dòng là điểm của 1 bug với tất cả các file

In [15]:
def get_sorted_ids_by_time(bug_reports):
    # Sắp xếp src_files theo report_time và lấy các id
    sorted_ids = sorted(bug_reports.items(), key=lambda item: item[1].report_time)
    return [item[0] for item in sorted_ids]

In [16]:
id_rp2_index = {}

In [17]:
count = 0
for report_id in bug_reports.keys():
    id_rp2_index[report_id] = count
    count += 1
print(id_rp2_index)

{'56012': 0, '56029': 1, '55943': 2, '56025': 3, '56010': 4, '55996': 5, '55995': 6, '55893': 7, '55978': 8, '55976': 9, '55974': 10, '55970': 11, '55960': 12, '55958': 13, '55937': 14, '55939': 15, '55905': 16, '55884': 17, '55820': 18, '55811': 19, '55801': 20, '55851': 21, '55842': 22, '55837': 23, '55804': 24, '55798': 25, '55778': 26, '55776': 27, '55772': 28, '55760': 29, '55749': 30, '55735': 31, '55715': 32, '55702': 33, '55684': 34, '55691': 35, '55681': 36, '55664': 37, '55656': 38, '54095': 39, '55251': 40, '55642': 41, '55630': 42, '55629': 43, '55317': 44, '55576': 45, '55582': 46, '55570': 47, '46727': 48, '52558': 49, '55552': 50, '54693': 51, '55521': 52, '55527': 53, '55343': 54, '55342': 55, '55500': 56, '55494': 57, '55453': 58, '51526': 59, '55469': 60, '55454': 61, '55442': 62, '55438': 63, '55434': 64, '55354': 65, '55357': 66, '55400': 67, '55404': 68, '52092': 69, '55345': 70, '55333': 71, '55316': 72, '55312': 73, '55309': 74, '55287': 75, '55267': 76, '55291':

In [16]:
bug_rp_ids =get_sorted_ids_by_time(bug_reports)
src_file_name = list(src_files.keys())

print(bug_rp_ids)

['11280', '28919', '29186', '29662', '29665', '29769', '29934', '29959', '30026', '30168', '30439', '30663', '31423', '31460', '32239', '32421', '32463', '33474', '33635', '33948', '34820', '34858', '34925', '34951', '35725', '36046', '36234', '36430', '36564', '36803', '36936', '37021', '37739', '38131', '38717', '38824', '39436', '39458', '39626', '39842', '39959', '39993', '40192', '40257', '40380', '40534', '40771', '41123', '41170', '41254', '41359', '41888', '41952', '42515', '42539', '42573', '42740', '42993', '43033', '43194', '43709', '44191', '44272', '44586', '44587', '45489', '46280', '46298', '47318', '47754', '47910', '47952', '48072', '48091', '48990', '49250', '49457', '49638', '49657', '49743', '49814', '50200', '50570', '50776', '51320', '51322', '51929', '52394', '52928', '53012', '53981', '53999', '54421', '54625', '54965', '55134', '55341', '56779', '57430', '57436', '57666', '58520', '58524', '58681', '59076', '59196', '59208', '59596', '59599', '59909', '60015', 

In [33]:
print(bug_reports['10526'].report_time)
print(bug_reports['10972'].report_time)
print(bug_reports['11117'].report_time)

2002-07-06 13:11:40
2002-07-19 02:30:07
2002-07-24 07:44:57


In [17]:
def get_days_between(d1, d2):
    """Calculates the number of days between two date objects

    Arguments:
        d1 {datetime} -- date 1
        d2 {datetime} -- date 2
    """
    diff_in_days = abs((d1 - d2).days)
    return diff_in_days

In [18]:
def previous_reports(filename, bug_rp_ids,report, bug_reports):
    """ Returns a list of previously filed bug id reports that share a file with the current bug report

    Arguments:
        filename {string} -- the name of the shared Java file
        bug_rp_ids  -- list bug report ids train sorted by report_time 
        report -- current bug report
        bug_reports  -- current bug report id
    """
    previous_reports = []
    if report in bug_rp_ids:
        index = bug_rp_ids.index(report)

        for id_bug in bug_rp_ids[:index]:
            if filename in bug_reports[id_bug].fixed_files:
                previous_reports.append(id_bug)


    return previous_reports

In [19]:
def bug_fixing_recency(curr_bug_id, prev_reports, bug_reports):
    """ Calculates the Bug Fixing Recency as defined by Lam et al.

    Arguments:
        curr_bug_id {string} -- current bug report id to get current time
        prev_reports {list of string} -- all previous bug reports id fixed that file
        bug_reports {dictionary} -- all bug reports
    """
    if len(prev_reports) > 0:
        recent_report = max(prev_reports, key=lambda x: bug_reports[x].report_time)
        
    else :
        return 1
    
    if curr_bug_id and recent_report:
        return 1 / float(
            get_days_between(bug_reports[curr_bug_id].report_time, bug_reports[recent_report].report_time) + 1
        )

    return 0

In [34]:
def bug_simi(bug_reports):
    """ Calculates the Bug Similarity as defined by Lam et al.

    Arguments:
        bug_reports {dictionary} -- all bug reports
    """
    bug_simis = []
    for report1 in bug_reports.keys():
        bug_simi = []
        for report2 in bug_reports.keys():
            
                
            bug_simi.append(cos_simi(get_all_text_report(report1, bug_reports), get_all_text_report(report2, bug_reports)))
        bug_simis.append(bug_simi)
    # Chuyển đổi danh sách thành DataFrame
    bug_simi_df = pd.DataFrame(bug_simis, index=bug_reports.keys(), columns=bug_reports.keys())
    print(bug_simi_df.shape)
    bug_simi_df.to_csv("bug_simi_"+dataset+".csv")
    

In [30]:
bug_simi(bug_reports)

(1016, 1016)


In [20]:
bug_simi_df = pd.read_csv("bug_simi_"+dataset+".csv", index_col=0)
bug_simi_df.index = bug_simi_df.index.astype(str)


In [21]:
def simi_previous_report(bug_id, prev_reports, bug_reports,bug_simi_df):
    """[summary]

    Arguments:
        bug_id {string} -- bug id of curent report
        prev_reports {list} -- list of previous reports id fix that file
    """
    
    sims = []
    for prev_report in prev_reports:
        sims.append((prev_report,bug_simi_df.at[bug_id, prev_report]))
    sims_sorted = sorted(sims, key=lambda x: x[1], reverse=True)
    if(len(prev_reports)<3):
        k=len(prev_reports)
    else:
        k=3
    score = 0.0
    for i, (_, sim) in enumerate(sims_sorted[:k]):
        score += sim / (i + 1)  # i+1 để tránh chia cho 0
    return score


In [22]:
def stack_trace (report, file_name, bug_reports):
    """[summary]

    Arguments:
        report {string} -- bug report id
        bug_reports {dictionary} -- all bug reports
        src_files {dictionary} -- all source files
    """
    stack_trace = bug_reports[report].stack_traces
    #print(stack_trace)
    for file,func in stack_trace:
        parts = file.split('.')
        # Lấy toàn bộ phần trước tên hàm
        class_path = '.'.join(parts[:-1])  # bỏ tên hàm
        class_path = class_path.replace('.', os.sep) + '.java'
        #print(class_path)

        if file_name.endswith(class_path):
            return 1
    return 0

In [49]:
stack_trace('54807','java\\org\\apache\\tomcat\\websocket\\server\\WsServerContainer.java',bug_reports)

1

In [23]:
def check_token_matchings(src_files, bug_reports):
    scores = []

    for report in bug_reports.values():
        matched_count = []
        summary_tokens = set(report.summary['stemmed'])
        pos_tokens = set(report.summary['stemmed'] + report.pos_tagged_description['stemmed'])

        for src in src_files.values():
            file_tokens = set(src.file_name['stemmed'])
            common = len(summary_tokens & file_tokens)
            matched_count.append(common)

        # Nếu không có file nào khớp với tên tóm tắt
        if sum(matched_count) == 0:
            matched_count = []
            for src in src_files.values():
                tokens = set(src.file_name['stemmed'] +
                             src.class_names['stemmed'] +
                             src.method_names['stemmed'] +
                             src.comments['stemmed'] +
                             src.attributes['stemmed'])
                common = len(pos_tokens & tokens)
                matched_count.append(common)

        # Chuẩn hóa
        
        scores.append(matched_count)

    return scores


In [35]:
token_simi = check_token_matchings(src_files, bug_reports)
df_token = pd.DataFrame(token_simi, index=[str(x) for x in bug_reports.keys()], columns=src_files.keys())
df_token.to_csv("token_scores_"+dataset+".csv")

In [24]:
df_token = pd.read_csv("token_scores_"+dataset+".csv", index_col=0)
df_token.index = df_token.index.astype(str)
if 'org.aspectj.ajdt.core\\src\\org\\aspectj\\ajdt\\internal\\compiler\\ast\\InterSuperFixerVisitor.java' in df_token.columns:
    print("ok")

ok


In [25]:
def class_simi_token(src_files, bug_reports):
    """
    Tính ma trận tương đồng giữa bug report và file mã nguồn dựa trên độ dài
    của tên class dài nhất có xuất hiện trong bug report.

    Args:
        src_files (dict): từ điển chứa thông tin source file, mỗi phần tử có field class_names['original'].
        bug_reports (dict): từ điển chứa thông tin bug report, mỗi phần tử có summary và description.

    Returns:
        pd.DataFrame: ma trận tương đồng, index là bug ID, columns là source file.
    """
    sim_matrix = []

    for bug_id, report in bug_reports.items():
        bug_words = set(report.summary['unstemmed'] + report.description['unstemmed'])

        row_scores = []
        for src_id, src in src_files.items():
            class_names = src.class_names['unstemmed']  # danh sách tên class gốc
            matched_names = [cn for cn in class_names if cn in bug_words]
            score = max([len(cn) for cn in matched_names], default=0)
            row_scores.append(score)
        
        sim_matrix.append(row_scores)

    df_sim = pd.DataFrame(sim_matrix, 
                          index=[str(bug_id) for bug_id in bug_reports.keys()],
                          columns=src_files.keys())
    df_sim.to_csv("class_token_simi_"+dataset+".csv")
    


In [38]:
class_simi_token(src_files, bug_reports)

In [26]:
df_class_token_simi = pd.read_csv("class_token_simi_"+dataset+".csv", index_col=0)
df_class_token_simi.index = df_class_token_simi.index.astype(str)

In [27]:
def splitKfold(all_bug_ids,k): # train on fold indexStart, test on fold indexStart+1
    fold_size = len(all_bug_ids) // k  # Kích thước fold
    folds = []
    for i in range(k):
        start = i * fold_size
        end = (i + 1) * fold_size if i != k - 1 else len(all_bug_ids)
        folds.append(all_bug_ids[start:end])
    return folds

In [82]:
folds = splitKfold(bug_rp_ids,3)


In [28]:
def top_k_wrong_file(report,bug_reports,src_files,df_rVSM):
    """ Returns the top k wrong files from pool random for a bug report

    Arguments:
        report {object} -- bug report
        bug_rp_ids_train {list} -- list of all bug report ids in the training set
        k {int} -- number of top files to return
    """
    # Get the top k wrong files for the current bug report
    
    all_files = []
    right_files = bug_reports[report].fixed_files
    
    for filename in list(set(src_files.keys()) - set(right_files)):
        all_files.append([filename,df_rVSM.at[report,filename]])
    # Sort the files by their similarity scores in descending order
    all_files.sort(key=lambda x: x[1], reverse=True)
    if (len(src_files)>1000):
        return [filename for filename, _ in all_files[:1000]]
    else :
        return list(set(src_files.keys()) - set(right_files))
    

In [29]:
def class_simi(src_files, bug_reports):
    src_class_strings = [' '.join(src.class_names['stemmed']) for src in src_files.values()]
    report_strings = [' '.join(report.summary['stemmed'] + report.description['stemmed']) 
                      for report in bug_reports.values()]
    
    tfidf = TfidfVectorizer(sublinear_tf=True, smooth_idf=True)
    all_docs = report_strings + src_class_strings
    tfidf_matrix = tfidf.fit_transform(all_docs)

    # Chia ma trận TF-IDF
    report_vecs = tfidf_matrix[:len(report_strings)]
    src_vecs = tfidf_matrix[len(report_strings):]

    # Tính cosine similarity giữa từng report và từng source file
    sim_matrix = cosine_similarity(report_vecs, src_vecs)

    # Tạo DataFrame
    df_sim = pd.DataFrame(sim_matrix, 
                          index=[str(k) for k in bug_reports.keys()],
                          columns=src_files.keys())
    
    return df_sim

In [ ]:
df_class_simi = class_simi(src_files,bug_reports )
#df_class_simi = pd.DataFrame(class_simi_score, index=[str(x) for x in bug_reports.keys()], columns=src_files.keys())
df_class_simi.to_csv("class_scores_"+dataset+".csv")

In [30]:
df_class_simi = pd.read_csv("class_scores_"+dataset+".csv", index_col=0)
df_class_simi.index = df_class_simi.index.astype(str)
class_simi_score = df_class_simi.values.tolist()
df_class_simi.shape

(562, 6910)

In [31]:
def create_data_train(src_files,bug_reports, bug_report_id_train, df_token, df_rVSM,df_class_simi,bug_simi_df,df_class_token_simi,up_sample = 1):
    # bug_report_ids là danh sách các id của bug reports đã qua sort
    # tạm thời đang có: semantic_score, rVSM_score, prev_rp_simi, prev_rp_recency, num_fixing
    #prev_rp_simi: so sánh độ tương đồng của rp với các rp trước đó đã fix file này
    #prev_rp_recency: thời gian từ lần gần nhất fix
    #num_fixing: số lần đã fix file này trước đó
   
    X_pos=  []
    X_neg =  []
    for report in bug_report_id_train:
        #them case với label true
        for fixed_file_name in bug_reports[report].fixed_files:
            semantic_score = df_semantic.at[report,fixed_file_name]
            rVSM_sc = df_rVSM.at[report,fixed_file_name]
            prev_rps = previous_reports(fixed_file_name, bug_rp_ids, report, bug_reports)
            prev_rp_simi = simi_previous_report(report, prev_rps, bug_reports,bug_simi_df)
            prev_rp_recency = bug_fixing_recency(report, prev_rps, bug_reports)
            stack_trace_score = stack_trace(report, fixed_file_name, bug_reports)
            token_simi_score = df_token.at[report,fixed_file_name]
            class_simi = df_class_simi.at[report,fixed_file_name]
            class_token_simi = df_class_token_simi.at[report,fixed_file_name]
            num_fixing = len(prev_rps)
            for i in range(up_sample):
                #X.append([report,fixed_file_name, rVSM_sc, prev_rp_simi,stack_trace_score,token_simi_score,class_simi,num_fixing,prev_rp_recency])
                X_pos.append([report,fixed_file_name, rVSM_sc, prev_rp_simi,class_simi,stack_trace_score,token_simi_score,prev_rp_recency,class_token_simi,semantic_score,num_fixing,1])
                
            #X.append([report,fixed_file_name,semantic_score, rVSM_sc, prev_rp_simi, prev_rp_recency, num_fixing])
            
            #y.append(1)
        #thêm case với label false
        #wrong_files = list(set(src_files.keys()) - set(bug_reports[report].fixed_files))
        wrong_files  = top_k_wrong_file(report,bug_reports,src_files,df_rVSM)
        for wrong_file_name in wrong_files:
            semantic_score = df_semantic.at[report,wrong_file_name]
            rVSM_sc = df_rVSM.at[report,wrong_file_name]
            prev_rps = previous_reports(wrong_file_name, bug_rp_ids, report, bug_reports)
            prev_rp_simi = simi_previous_report(report, prev_rps, bug_reports,bug_simi_df)
            prev_rp_recency = bug_fixing_recency(report, prev_rps, bug_reports)
            class_token_simi = df_class_token_simi.at[report,wrong_file_name]
            stack_trace_score = stack_trace(report, wrong_file_name, bug_reports)
            token_simi_score = df_token.at[report,wrong_file_name]
            class_simi = df_class_simi.at[report,wrong_file_name]
            num_fixing = len(prev_rps)
            X_neg.append([report,wrong_file_name, rVSM_sc, prev_rp_simi,class_simi,stack_trace_score,token_simi_score,prev_rp_recency,class_token_simi,semantic_score,num_fixing,0])
            #X.append([report,wrong_file_name, rVSM_sc, prev_rp_simi,stack_trace_score,token_simi_score,class_simi,num_fixing,prev_rp_recency])
           
            
    return X_pos,X_neg

In [32]:
def create_test(report,bug_report_id_train,src_files,bug_reports, df_rVSM,df_token,df_class_simi,bug_simi_df,df_class_token_simi):
    '''
    Create test data for a specific bug report
    :param report: bug report id
    :param bug_report_id_train: list of bug report ids in the training set
    :param src_files: source files
    :param bug_reports: bug reports
    :param df_rVSM: rVSM scores
    :param df_semantic: semantic scores
    :return: test data for one bug report
    '''
    X_test = []
    for file_name in src_files.keys():
        semantic_score = df_semantic.at[report,file_name]
        rVSM_sc = df_rVSM.at[report,file_name]
        prev_rps = previous_reports(file_name, bug_rp_ids, report,bug_reports)
        prev_rp_simi = simi_previous_report(report, prev_rps, bug_reports,bug_simi_df)
        prev_rp_recency = bug_fixing_recency(report, prev_rps, bug_reports)
        class_token_simi = df_class_token_simi.at[report,file_name]
        stack_trace_score = stack_trace(report, file_name, bug_reports)
        token_simi_score = df_token.at[report,file_name]
        class_simi = df_class_simi.at[report,file_name]
        num_fixing = len(prev_rps)
        X_test.append([report,file_name, rVSM_sc, prev_rp_simi,class_simi,stack_trace_score,token_simi_score,prev_rp_recency,class_token_simi,semantic_score,num_fixing])
        
    return X_test

In [33]:
folds = splitKfold(bug_rp_ids, 3)
print(len(folds[0]),len(folds[1]))

187 187


In [54]:
bug_train = folds[0]
bug_test = folds[1]

In [55]:
print(bug_train)

['2035', '2172', '2345', '2369', '3970', '4922', '7235', '7712', '9262', '9826', '11983', '12116', '12329', '15844', '16179', '16552', '16854', '17256', '17480', '17826', '21013', '21357', '22563', '22615', '22698', '24024', '24143', '24157', '24168', '24521', '24586', '24928', '25387', '25419', '25458', '25985', '26109', '26133', '26221', '26366', '26823', '27408', '27455', '27530', '28508', '28623', '28638', '28785', '28831', '29107', '29163', '29609', '29633', '29853', '30170', '30344', '30402', '30610', '30751', '30866', '32038', '32053', '32147', '32198', '32254', '32262', '32553', '32610', '33085', '33488', '33550', '33628', '33897', '34421', '34528', '35192', '35666', '35917', '36141', '36446', '36525', '36526', '37155', '37253', '37277', '37787', '38009', '38049', '38072', '38085', '38092', '38247', '38462', '38547', '38729', '38782', '39003', '39074', '40559', '40638', '40965', '41589', '41743', '41927', '42009', '42056', '43415', '43873', '44162', '44393', '44443', '44563', '

In [74]:
X_train, y_train = create_label_train(src_files,bug_reports, bug_train, df_token, df_rVSM,df_class_simi,bug_simi_df,df_class_token_simi,pool=60,k=30, up_sample =10)

In [75]:
X_train, y_train = shuffle(X_train, y_train, random_state=42)
info_cols = X_train[:, :2]      # Lấy 2 cột đầu
X_train = X_train[:, 2:] # Lấy các cột còn lại

scaler = CustomMinMaxScaler()
X_train = scaler.fit_transform(X_train)

ValueError: could not convert string to float: '[0.4333060557997636]'

In [84]:
print(X_train[:5])  # In ra 5 mẫu đầu tiên
print(type(X_train[0]), type(X_train[0][0]))

[['[0.4333060557997636]' '0.0' '0.0' '0' '0' '1' '0.858651855313182']
 ['[0.03260908516060898]' '0.09422571457104997' '0.0' '0' '0' '1.0'
  '0.655347659364934']
 ['[0.0]' '0.0' '0.0' '0' '0' '1' '0.0']
 ['[0.0]' '0.0' '0.0' '0' '0' '1' '0.0']
 ['[0.062213502174300106]' '0.0' '0.0750038724896054' '0' '0' '1'
  '0.7390711831362589']]
<class 'numpy.ndarray'> <class 'numpy.str_'>


In [151]:
type(X_train[0][3])

numpy.float64

In [176]:
print(X_train.shape)
print(y_train.shape)

(8840, 7)
(8840,)


In [175]:
clf = MLPClassifier(
    hidden_layer_sizes=(300,),
    alpha=1e-5,
    solver='sgd',
    random_state=1,
    max_iter=10000,
    n_iter_no_change=30
)
clf.fit(X_train, y_train)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(300,), max_iter=10000,
              n_iter_no_change=30, random_state=1, solver='sgd')

In [218]:
clf = MLPClassifier(
    hidden_layer_sizes=(128, 64),
    activation='relu',
    solver='adam',
    alpha=1e-4,
    learning_rate_init=0.001,
    max_iter=10000,
    early_stopping=True,
    n_iter_no_change=30,
    random_state=1
)
clf.fit(X_train, y_train)

MLPClassifier(early_stopping=True, hidden_layer_sizes=(128, 64), max_iter=10000,
              n_iter_no_change=30, random_state=1)

In [35]:
def dnn_score(bug_trains,bug_tests,src_files,scaler):
    scores = []
    bug_ids_test = []
    for bug_id in bug_tests:
        
        X_test = np.array(create_test(bug_id, bug_trains, src_files, bug_reports, df_rVSM, df_token, df_class_simi,bug_simi_df,df_class_token_simi))
        X_test = X_test[:, 2:].astype(np.float32)
        
        X_test = scaler.transform(X_test)
        
        y_pred = clf.predict_proba(X_test)[:, 1]
            
        scores.append(y_pred.flatten().tolist())
        bug_ids_test.append(bug_id)
    return scores, bug_ids_test


In [36]:
def dnn_score_keras(bug_trains, bug_tests, src_files, scaler, model):
    scores = []
    bug_ids_test = []
    
    for bug_id in bug_tests:
        # Tạo dữ liệu kiểm tra từ bug_reports, bug_trains, src_files
        X_test = np.array(create_test(bug_id, bug_trains, src_files, bug_reports, df_rVSM, df_token, df_class_simi, bug_simi_df, df_class_token_simi))
        
        # Chọn các cột từ 2 trở đi và chuyển sang kiểu dữ liệu float32
        X_test = X_test[:, 2:].astype(np.float32)
        
        # Chuẩn hóa dữ liệu kiểm tra
        X_test = scaler.transform(X_test)
        
        # Dự đoán xác suất với mô hình DNN (sử dụng Keras model)
        y_pred = model.predict(X_test,verbose=0)
        
        # Thêm kết quả dự đoán vào danh sách scores
        scores.append(y_pred.flatten().tolist())
        
        # Thêm bug_id vào danh sách bug_ids_test
        bug_ids_test.append(bug_id)
    
    return scores, bug_ids_test

In [34]:
def evaluate_test(src_files, bug_test, scores, top_n=(1, 5, 10)):
    top_n_rank = [0] * len(top_n)
    mrr = []
    ap_scores = []

    for i, bug_id in enumerate(bug_test):
        # Sắp xếp source files theo điểm dự đoán
        src_ranks, _ = zip(*sorted(zip(src_files.keys(), scores[i]),
                                   key=operator.itemgetter(1), reverse=True))

        fixed_files = bug_reports[bug_id].fixed_files
        if not fixed_files:
            mrr.append(0)
            ap_scores.append(0)
            continue

        # Tính top-n hit
        for k, rank in enumerate(top_n):
            hit = set(src_ranks[:rank]) & set(fixed_files)
            if hit:
                top_n_rank[k] += 1

        # Tính các vị trí đúng (relevant ranks)
        relevant_ranks = []
        for fixed in fixed_files:
            if fixed in src_ranks:
                relevant_ranks.append(src_ranks.index(fixed) + 1)  # +1 vì xếp hạng bắt đầu từ 1

        if not relevant_ranks:
            mrr.append(0)
            ap_scores.append(0)
            continue

        # MRR
        min_rank = min(relevant_ranks)
        mrr.append(1 / min_rank)

        # AP (Average Precision)
        num_hits = 0
        sum_precisions = 0
        for rank_idx, file in enumerate(src_ranks):
            if file in fixed_files:
                num_hits += 1
                precision_at_k = num_hits / (rank_idx + 1)
                sum_precisions += precision_at_k
        ap = sum_precisions / len(fixed_files)
        ap_scores.append(ap)

    mrr_score = np.mean(mrr)
    map_score = np.mean(ap_scores)
    top_n_score = [x / len(bug_test) for x in top_n_rank]
    return top_n_score, mrr_score, map_score

In [178]:
score_dnn, bug_id_tests = dnn_score(bug_train,bug_test,src_files,scaler)

In [179]:
top_n = (1,2,3,4, 5, 10,15)
top_n_score,mrr_score = evaluate_test(src_files,bug_id_tests,score_dnn,top_n)
for i,score in zip(top_n,top_n_score):
    print(f"Top{i}: {score}")
print(mrr_score)

Top1: 0.4630541871921182
Top2: 0.5517241379310345
Top3: 0.5812807881773399
Top4: 0.6108374384236454
Top5: 0.6600985221674877
Top10: 0.7438423645320197
Top15: 0.7733990147783252
0.5531759725911535


In [37]:
import tensorflow as tf

def focal_loss(gamma=2., alpha=0.75):
    def focal_loss_fixed(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        
        # Tính p_t: xác suất dự đoán cho lớp đúng
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        
        # Cross entropy
        cross_entropy = -tf.math.log(p_t)
        
        # Focal weight
        focal_weight = tf.pow(1 - p_t, gamma)
        
        # Focal loss với alpha
        loss = alpha * focal_weight * cross_entropy
        
        return tf.reduce_mean(loss)
    return focal_loss_fixed

In [38]:
def dnn_score_keras_drop(bug_trains, bug_tests, src_files, scaler, model,i_):
    scores = []
    bug_ids_test = []
    
    for bug_id in bug_tests:
        # Tạo dữ liệu kiểm tra từ bug_reports, bug_trains, src_files
        X_test = np.array(create_test(bug_id, bug_trains, src_files, bug_reports, df_rVSM, df_token, df_class_simi, bug_simi_df, df_class_token_simi))
        
        # Chọn các cột từ 2 trở đi và chuyển sang kiểu dữ liệu float32
        X_test = X_test[:, 2:].astype(np.float32)
        if(len(i_)>0):
            
                # Xóa cột thứ i trong X_test
            X_test = np.delete(X_test, i_, axis=1)
        # Chuẩn hóa dữ liệu kiểm tra
        X_test = scaler.transform(X_test)
        
        # Dự đoán xác suất với mô hình DNN (sử dụng Keras model)
        y_pred = model.predict(X_test,verbose=0)
        
        # Thêm kết quả dự đoán vào danh sách scores
        scores.append(y_pred.flatten().tolist())
        
        # Thêm bug_id vào danh sách bug_ids_test
        bug_ids_test.append(bug_id)
    
    return scores, bug_ids_test

In [39]:
def calculate_batch_params(Nneg, Npos, desired_batch_size=128, desired_ratio=None):
    """
    Tự động tính K, Sn, Sp dựa trên dữ liệu.
    
    Args:
        Nneg (int): Số mẫu âm tính.
        Npos (int): Số mẫu dương tính.
        desired_batch_size (int): Kích thước mini-batch mong muốn.
        desired_ratio (float): Tỷ lệ Sn:Sp mong muốn. Nếu None, dùng tỷ lệ gốc.
    
    Returns:
        tuple: (K, Sn, Sp)
    """
    total_samples = Nneg + Npos
    K = max(5, math.ceil(total_samples / desired_batch_size))  # Tối thiểu 5 mini-batches
    Sn = math.floor(Nneg / K)
    if Sn < 10:
        K = max(5, math.floor(Nneg / 10))
        Sn = math.floor(Nneg / K)
    
    if desired_ratio is None:
        desired_ratio = Nneg / Npos if Npos > 0 else 10
    Sp = math.floor(Sn / desired_ratio)
    if Sp < 1:
        Sp = 1
    
    return K, Sn, Sp

In [56]:
print(calculate_batch_params(len(folds[0])*len(src_files), len(folds[0])*2+3, desired_batch_size=128, desired_ratio=10))

(2849, 127, 12)


In [40]:
def create_mini_batches(negative_set, positive_set, Sn, Sp):
    """
    Tạo K mini-batches cho DNN bằng bootstrapping.
    
    Args:
        negative_set (list): Tập âm tính với Nneg mẫu (tệp không chứa lỗi).
        positive_set (list): Tập dương tính với Npos mẫu (tệp chứa lỗi).
        K (int): Số mini-batches.
        Sn (int): Số mẫu âm tính mỗi mini-batch.
        Sp (int): Số mẫu dương tính mỗi mini-batch.
    
    Returns:
        list: Danh sách K mini-batches, mỗi mini-batch là danh sách [âm tính, dương tính].
    """
    
    Nneg = len(negative_set)
    Npos = len(positive_set)
    K = math.ceil(Nneg / Sn)
    if Nneg < K * Sn:
        K = K-1
    if Npos < 1:
        raise ValueError("Tập dương tính rỗng")

    # Bước 1: Chia tập âm tính
    negative_indices = list(range(Nneg))
    random.shuffle(negative_indices)
    X = [negative_indices[i * Sn:(i + 1) * Sn] for i in range(K)]
    X = [[negative_set[idx] for idx in subset] for subset in X]
    
    # Bước 2-3: Thêm Sp mẫu dương tính với sampling with replacement
    for i in range(K):
        posi = []
        for j in range(Sp):
            t = random.choice(positive_set)  # Chọn ngẫu nhiên, không xóa
            posi.append(t)
        X[i].extend(posi)
    
    return X

In [41]:
def prepare_data_for_dnn(mini_batches):
    """
    Chuyển mini-batches thành dữ liệu và nhãn, lấy nhãn từ thuộc tính cuối.
    
    Args:
        mini_batches (list): Danh sách K mini-batches, mỗi mẫu có nhãn ở cuối.
    
    Returns:
        tuple: (X_train, y_train)
    """
    X_train = []
    y_train = []
    
    for batch in mini_batches:
        # Chuyển batch thành mảng NumPy
        batch_array = np.array(batch)
        # Tách đặc trưng (tất cả trừ cột cuối) và nhãn (cột cuối)
        batch_features = batch_array[:, :-1]  # [Sn + Sp, feature_dim]
        batch_labels = batch_array[:, -1]    # [Sn + Sp]
        X_train.append(batch_features)
        y_train.append(batch_labels)
        
    
    # Gộp thành mảng
    X_train = np.vstack(X_train)  # [K * (Sn + Sp), feature_dim]
    y_train = np.concatenate(y_train)  # [K * (Sn + Sp)]
    y_train = y_train.astype(int)
    # Kiểm tra nhãn
    
    
    return X_train, y_train

In [ ]:
X_pos, X_neg = create_data_train(src_files,bug_reports, bug_train, df_token, df_rVSM,df_class_simi,bug_simi_df,df_class_token_simi,up_sample =1)
mini_batchs = create_mini_batches(X_neg, X_pos, Sn=120, Sp=8)


In [ ]:
X_train, y_train = prepare_data_for_dnn(mini_batchs)
y_train = y_train.astype(int)

In [81]:
print(X_train[0])

['48689' 'java\\org\\apache\\catalina\\websocket\\WsOutbound.java'
 '0.0552577443927334' '0.0' '0.0' '0' '0.0' '1' '0.0' '0.4991143894270685'
 '0']


In [96]:
folds = splitKfold(bug_rp_ids, 3)
top_n_score_dnns = []
mrrs = []
map_scores = []

In [43]:
chooses = [[0, 1, 2, 3, 4, 5, 7, 8]]
for choose in chooses:
    top_n_score_dnns = []
    mrrs = []
    map_scores = []

    for i in range(len(folds)-1):

        #
        
        bug_train = folds[i]
        bug_test = folds[i+1]
        X_pos, X_neg = create_data_train(src_files,bug_reports, bug_train, df_token, df_rVSM,df_class_simi,bug_simi_df,df_class_token_simi,up_sample =1)
        BATCH_SIZE = 16
        Sn = 14
        Sp = BATCH_SIZE - Sn
        mini_batchs = create_mini_batches(X_neg, X_pos, Sn=Sn, Sp=Sp)
        X_train, y_train = prepare_data_for_dnn(mini_batchs)
        
        info_cols = X_train[:, :2]      # Lấy 2 cột đầu
        X_train = X_train[:, 2:] # Lấy các cột còn lại
        #[ rVSM_sc, prev_rp_simi,class_simi,stack_trace_score,token_simi_score,prev_rp_recency,class_token_simi,semantic_score]
        dele = [i for i in range(9) if i not in choose]
        if len(dele) > 0:
            X_train = np.delete(X_train, dele, axis=1)
        scalerMinMax = CustomMinMaxScaler()
        X_train = scalerMinMax.fit_transform(X_train)
        '''
        clf = MLPClassifier(
        hidden_layer_sizes=(300,150),
        alpha=1e-5,
        solver='sgd',
        random_state=1,
        max_iter=10000,
        n_iter_no_change=30
        )
        
        
        clf.fit(X_train, y_train)
        score_dnn, bug_id_tests = dnn_score(bug_train,bug_test,src_files,scalerMinMax,clf)
        '''
        model = Sequential()

        # First hidden layer (300 nodes, ReLU activation)
        model.add(Dense(300, activation='relu', input_dim=X_train.shape[1]))  # 'input_dimension' should be the number of input features
        
        # Second hidden layer (150 nodes, ReLU activation)
        model.add(Dense(150, activation='relu'))
        #model.add(Dropout(0.2))
        # Output layer (1 node, sigmoid activation)
        model.add(Dense(1, activation='sigmoid'))

        # Define the optimizer (SGD)
        optimizer = SGD(learning_rate=0.01)

        # Compile the model with binary cross-entropy loss
        model.compile(optimizer=optimizer, loss=focal_loss(gamma=2.5, alpha=0.99), metrics=['accuracy'])

        # Train the model
        model.fit(X_train, y_train, epochs=30, batch_size=BATCH_SIZE,verbose=1)
        # Evaluate the model on the training data
        score_dnn, bug_id_tests = dnn_score_keras_drop(bug_train,bug_test,src_files,scalerMinMax,model,dele)
        top_n = (1,2,3,4, 5, 10,15)
        top_n_score,mrr_score,map_score = evaluate_test(src_files,bug_id_tests,score_dnn,top_n)

        top_n_score_dnns.append(top_n_score)
        mrrs.append(mrr_score)
        map_scores.append(map_score)
        print(f"fold {i+1}")
        for i,score in zip(top_n,top_n_score):
            print(f"Top{i}: {score}")
        print(mrr_score)
        print(map_score)
        print("__________________________________")
    top_n_score_mean = [mean(col) for col in zip(*top_n_score_dnns)]
    print("__________________________________")
    print("ALL FOLDS "+dataset)
    print(choose)
    for i,score in zip(top_n,top_n_score_mean):
        print(f"Top{i}: {score}")
    print(mean(mrrs))
    print(mean(map_scores))
    print("__________________________________")


Epoch 1/30


c:\Users\xuand\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13357/13357 ━━━━━━━━━━━━━━━━━━━━ 17s 1ms/step - accuracy: 0.8821 - loss: 0.0622
Epoch 2/30
13357/13357 ━━━━━━━━━━━━━━━━━━━━ 17s 1ms/step - accuracy: 0.8972 - loss: 0.0569
Epoch 3/30
13357/13357 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - accuracy: 0.9006 - loss: 0.0554
Epoch 4/30
13357/13357 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - accuracy: 0.9023 - loss: 0.0538
Epoch 5/30
13357/13357 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - accuracy: 0.9016 - loss: 0.0533
Epoch 6/30
13357/13357 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - accuracy: 0.9002 - loss: 0.0526
Epoch 7/30
13357/13357 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - accuracy: 0.9018 - loss: 0.0510
Epoch 8/30
13357/13357 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - accuracy: 0.9017 - loss: 0.0502
Epoch 9/30
13357/13357 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - accuracy: 0.9026 - loss: 0.0491
Epoch 10/30
13357/13357 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - accuracy: 0.9025 - loss: 0.0489
Epoch 11/30
13357/13357 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - accuracy: 0.9035 - loss: 0.0482
Epoch 12

In [70]:
top_n_score,mrr_score,map_score = evaluate_test(src_files,bug_id_tests,score_dnn,top_n)
for i,score in zip(top_n,top_n_score):
    print(f"Top{i}: {score}")
print(mrr_score)
print(map_score)


Top1: 0.5018181818181818
Top2: 0.6036363636363636
Top3: 0.68
Top4: 0.7127272727272728
Top5: 0.7527272727272727
Top10: 0.8290909090909091
Top15: 0.850909090909091
0.609935763325994
0.5280297568532109


In [52]:
for j in range(len(top_n_score_dnns)):
    for i,score in zip(top_n,top_n_score_dnns[j]):
        print(f"Top{i}: {score}")
    print(mrrs[j])
    print(map_scores[j])
#print(mean(mrrs))

Top1: 0.49454545454545457
Top2: 0.6181818181818182
Top3: 0.6836363636363636
Top4: 0.7163636363636363
Top5: 0.7563636363636363
Top10: 0.8363636363636363
Top15: 0.8618181818181818
0.6110868644870419
0.5302201802243535
Top1: 0.4290909090909091
Top2: 0.5818181818181818
Top3: 0.6436363636363637
Top4: 0.68
Top5: 0.7127272727272728
Top10: 0.7927272727272727
Top15: 0.8218181818181818
0.5587284133860112
0.5042992924957339
Top1: 0.5163636363636364
Top2: 0.6327272727272727
Top3: 0.6763636363636364
Top4: 0.7236363636363636
Top5: 0.7418181818181818
Top10: 0.8181818181818182
Top15: 0.8545454545454545
0.6208513544113337
0.5486462131867073
Top1: 0.5381818181818182
Top2: 0.6618181818181819
Top3: 0.7454545454545455
Top4: 0.7636363636363637
Top5: 0.7818181818181819
Top10: 0.84
Top15: 0.8581818181818182
0.6483323969202507
0.5445866356374912
Top1: 0.43636363636363634
Top2: 0.5672727272727273
Top3: 0.6145454545454545
Top4: 0.6472727272727272
Top5: 0.6836363636363636
Top10: 0.7527272727272727
Top15: 0.807272

In [100]:
for j in range(len(top_n_score_dnns)):
    for i,score in zip(top_n,top_n_score_dnns[j]):
        print(f"Top{i}: {score}")
    print(mrrs[j])
print(mean(mrrs))

Top1: 0.1925133689839572
Top2: 0.2620320855614973
Top3: 0.3422459893048128
Top4: 0.39037433155080214
Top5: 0.42780748663101603
Top10: 0.5454545454545454
Top15: 0.5935828877005348
0.30086631755887605
0.30086631755887605


In [ ]:
src_rank_tests = zip(*sorted(zip(src_files.keys(), score_dnn[0]),
                                   key=operator.itemgetter(1), reverse=True))
for file, score in zip(*src_rank_tests):
    print(file, score)

In [37]:
src_ranks_temp, _ = zip(*sorted(zip(src_files.keys(), score_dnn[0]),
                                   key=operator.itemgetter(1), reverse=True))
# Getting reported fixed files
fixed_files = bug_reports[bug_id_tests[0]].fixed_files

# Iterating over top n
print(src_ranks_temp)
src_ranks_temp2, _ = zip(*sorted(zip(src_files.keys(), rVSM_score[id_rp2_index[bug_id_tests[0]]]),
                                   key=operator.itemgetter(1), reverse=True))
print(src_ranks_temp2)
print(fixed_files)

('java\\org\\apache\\coyote\\AsyncContextCallback.java', 'java\\org\\apache\\tomcat\\websocket\\server\\WsContextListener.java', 'java\\org\\apache\\catalina\\connector\\CoyoteOutputStream.java', 'webapps\\examples\\WEB-INF\\classes\\websocket\\tc7\\echo\\EchoStream.java', 'java\\org\\apache\\coyote\\http11\\upgrade\\NioServletOutputStream.java', 'webapps\\examples\\WEB-INF\\classes\\websocket\\echo\\EchoEndpoint.java', 'java\\org\\apache\\catalina\\manager\\Constants.java', 'java\\org\\apache\\catalina\\loader\\StandardClassLoaderMBean.java', 'java\\org\\apache\\coyote\\http11\\upgrade\\UpgradeBioProcessor.java', 'java\\org\\apache\\coyote\\http11\\upgrade\\UpgradeAprProcessor.java', 'java\\org\\apache\\coyote\\http11\\upgrade\\BioServletOutputStream.java', 'java\\org\\apache\\coyote\\http11\\upgrade\\UpgradeOutbound.java', 'java\\javax\\servlet\\descriptor\\JspConfigDescriptor.java', 'java\\org\\apache\\tomcat\\websocket\\pojo\\PojoEndpointClient.java', 'java\\org\\apache\\catalina\\